In [1]:
# Goal is to get the data from the csv files and put it all into one database
import numpy as np
import pandas as pd
import time
import matplotlib.pyplot as plt
import psycopg2
import scraper_tools

In [2]:
# And connect to the database we'll use
def connect_to_db():
    try:
        conn = psycopg2.connect("dbname='mypuzzle' user='postgres' host='localhost' password='gwv251'")
        cursor = conn.cursor()
        print("Connected to database")
    except:
        print("Unable to connect to database!")
    return conn
conn = connect_to_db()

Connected to database


In [3]:
#UPLOAD URLS
product_urls = np.loadtxt('1000_piece/product_urls.txt',dtype=str)

# Upload to db
# for url in product_urls:
#     query = """INSERT INTO public.products (product_url) 
#     VALUES  ('{0}')""".format(url)
#     cursor.execute(query)
# conn.commit()

In [121]:
# UPLOAD METADATA
reviews_df = scraper_tools.get_database()

n_products = reviews_df.shape[0]
# Fix some columns:
star_rating_num = np.zeros((n_products))
price_num = np.zeros((n_products))
n_reviews = np.zeros((n_products))
details = []
for ix in range(n_products):
    # Fix the star rating
    x = reviews_df['star_rating'][ix]
    if x != x: # Handle nans in shady way
        x = 0
    else:
        x = x[0:3]
    star_rating_num[ix] = float(x)

    # Fix the price
    price = reviews_df['Price'][ix]
    if price != price or len(price) <3:
        price = '$0.00'
    price = price.strip().replace('$','')
    price = float(price)
    price_num[ix] = price
    
    # Fix the customer reviews
    n_rev = reviews_df['n_reviews'][ix]
    if n_rev != n_rev:
        n_rev = -1
    else:
        n_rev = n_rev.strip().replace('customer review','').replace('s','')
        n_rev = int(n_rev)
    n_reviews[ix] = n_rev
    
    d = reviews_df['details'][ix]
    d = d.replace("', '",'. ').replace('[','').replace(']','').replace('..','')
    details.append(d)
# Save to db
for ix in range(n_products):
    row = reviews_df.iloc[ix]
    url = product_urls[ix]
    
    query = """UPDATE public.products 
    SET name = $${1}$$, n_reviews = {2}, price = {3}, brand = $${4}$$,
    img_url = $${5}$$, star_rating = $${6}$$, details = $${7}$$
    WHERE product_url = $${0}$$
    """.format(url,row['Name'],n_reviews[ix],price_num[ix],row['Brand'],
               row['img_url'],star_rating_num[ix],details[ix])
    cursor.execute(query)
conn.commit()

In [19]:
# Now save the reviews in the second table
import glob, os
review_files = glob.glob('1000_piece/product*.csv')

def clean_review_db(conn):
    print('Cleaning reviews table!')
    # Delete entries
    cur = conn.cursor()
    query = "DELETE FROM public.reviews"
    cur.execute(query)
    
    # Reset index
    query2 = """ALTER SEQUENCE reviews_review_index_seq RESTART WITH 1"""
    cur.execute(query2)
    
    conn.commit()

conn.close()
conn = connect_to_db()
cursor = conn.cursor()
clean_review_db(conn)
# Loop through 
for ix,url in enumerate(product_urls):
    review_file = '1000_piece/product_{0:05}.csv'.format(ix)
    if os.path.exists(review_file):
        df = pd.read_csv(review_file)
        reviews = df['Reviews']
        ratings = df['ReviewRatings']
    else:
        print('not found!',url)
        continue
        
    #Find out the product index
    query = """ SELECT products.product_index FROM public.products
        WHERE product_url = $${0}$$
        """.format(url)
    cursor.execute(query)
    prod_ix = cursor.fetchone()[0]
    
    # Save to the database!
    n_reviews = reviews.shape[0]
    for r_ix in range(n_reviews):
        # Clean the text a bit
        clean_text = reviews[r_ix]
        clean_text = clean_text.replace('$','')
        clean_text = clean_text.replace(':)','')
        
        query = """INSERT INTO public.reviews (product_index,review,rating) 
          VALUES  ({0},$${1}$$,{2})
           """.format(prod_ix,clean_text,ratings[r_ix])
        try:
            cursor.execute(query)
        except:
            print('bad query: ',prod_ix[0],ix,query)
            print(clean_text[r_ix])
    if (ix % 10) == 0:
        conn.commit() 

Connected to database
Cleaning reviews table!
not found! ['https://amazon.com/Buffalo-Games-Signature-Collection-Cinque/dp/B00ITTI3VW'
 'https://amazon.com/Buffalo-Games-Darrell-Jigsaw-Puzzle/dp/B00WOZBOGS'
 'https://amazon.com/Buffalo-Games-Marine-Bubbles-Jigsaw/dp/B07D8JKDRN'
 ...
 'https://amazon.com/Midwest-Products-9283-Nitsche-Searching/dp/B006LWPQ46'
 'https://amazon.com/Schmidt-SCH58151-Theresa-Jigsaw-1000-Piece/dp/B005JB9HOU'
 'https://amazon.com/Aquarius-Peace-Chance-Jigsaw-Puzzle/dp/B002WVD3HM']


In [20]:
print('Done')

Done
